goal of this notebook is to test the capabilities of the scipy.optimize module at a very, very basic level with some toy examples.

documentation is available at: https://docs.scipy.org/doc/scipy/tutorial/optimize.html

In [1]:
import scipy.optimize as op
import numpy as np

# 0.0: Optimize x^2

In [2]:
def x_squared(x):
    return x ** 2

In [3]:
res = op.minimize(x_squared, [1234], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 41
         Function evaluations: 82
[-9.09494702e-13]


comments: successfully finds f(0) = 0 as the minimum. this algorithm is, notably, not constrained.

# 0.1: Optimize x^2 + 5

In [4]:
def x_squared_add_five(x):
    return x ** 2 + 5

In [5]:
res = op.minimize(x_squared_add_five, [1234], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 5.000000
         Iterations: 41
         Function evaluations: 83
[-9.09494702e-13]


comments: again, finds f(0) = 5 as the minimum.

# 0.2: Optimize (x-7)^2

In [6]:
def x_minus_seven_squared(x):
    return (x - 7) ** 2

In [7]:
res = op.minimize(x_minus_seven_squared, [1234], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 41
         Function evaluations: 82
[7.]


comments: finds f(7) = 0 as the minimum.

# 0.3 Optimize (x-7) ^ 2, subject to x^2 < x

i am trying to use SLSQP here, which supports constraints that are relative to zero. that means, that equality constraints are of the from c(x) = 0, and ineq constraints are of the form c(x) > 0.

converting x^2 < x to this format yields: - x^2 + x > 0, whose first partial derivative is (-2x + 1)

the method needs the Jacobian of all the constraints, and it is to be provided as a dictionary (see example in the documentation: https://docs.scipy.org/doc/scipy/tutorial/optimize.html#sequential-least-squares-programming-slsqp-algorithm-method-slsqp)

In [8]:
ineq_cons = {'type': 'ineq',
             'fun' : lambda x: np.array([-x[0]**2 + x[0]]),
             'jac' : lambda x: np.array([-2.0 * x[0] + 1.0])
}

In [9]:
def x_minus_seven_squared_der(x):
    return 2 * (x - 7)


bounds = [(-1e5, 1e5)] # liberal bounds, not worried about this right now
res = op.minimize(x_minus_seven_squared, [1000], method='SLSQP', jac=x_minus_seven_squared_der,
               constraints=[ineq_cons], options={'ftol': 1e-9, 'disp': True},
               bounds=bounds)
print(res.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 35.999999999981064
            Iterations: 10
            Function evaluations: 12
            Gradient evaluations: 10
[1.]


comments: we correctly find the closest we can get to the minima (7) without messing up the constraint of (x ^ 2 < x)

# 1: Thinking About Parameters Defining Functions and Us Minimizing some Objective Function

In [28]:
def poly_3d(x, point_x):
    # x are the variables, point_x is the point the model is being evaluated at
    # polynomial of 3 degrees
    degree = 3
    result = 0
    
    for i in range(degree + 1): # [0, degree]
        result += x[i] * (point_x ** i)
    
    return result
    
def obj_function(x):
    # in a grid of points, evaluate how well the model did
    
    GOALS = [(px, 4*(px**2)) for px in range(-5, 5)] # f(x) = 4x^2. optimized by x=[0,0,4,0]
    score = 0
    for point_x, point_y in GOALS:
        print(point_y, poly_3d(x, point_x))
        score += abs(point_y - poly_3d(x, point_x))

                     
    return score

In [29]:
res = op.minimize(obj_function, [0,0,0,0], method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
print(res.x)

100 0.0
64 0.0
36 0.0
16 0.0
4 0.0
0 0.0
4 0.0
16 0.0
36 0.0
64 0.0
100 0.00025
64 0.00025
36 0.00025
16 0.00025
4 0.00025
0 0.00025
4 0.00025
16 0.00025
36 0.00025
64 0.00025
100 -0.00125
64 -0.001
36 -0.00075
16 -0.0005
4 -0.00025
0 0.0
4 0.00025
16 0.0005
36 0.00075
64 0.001
100 0.00625
64 0.004
36 0.0022500000000000003
16 0.001
4 0.00025
0 0.0
4 0.00025
16 0.001
36 0.0022500000000000003
64 0.004
100 -0.03125
64 -0.016
36 -0.00675
16 -0.002
4 -0.00025
0 0.0
4 0.00025
16 0.002
36 0.00675
64 0.016
100 0.033875
64 0.017625000000000002
36 0.007625
16 0.002375
4 0.000375
0 0.000125
4 0.000125
16 -0.0011250000000000001
36 -0.005124999999999999
64 -0.013375
100 0.0664375
64 0.0344375
36 0.0148125
16 0.004562500000000001
4 0.0006875
0 0.0001875
4 6.250000000000006e-05
16 -0.0026875
36 -0.0110625
64 -0.0280625
100 0.03771875
64 0.02034375
36 0.00940625
16 0.00340625
4 0.00084375
0 0.00021875
4 3.125000000000003e-05
16 -0.00121875
36 -0.00503125
64 -0.012906250000000001
100 0.055328125
64 0.0